In [18]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import plotly.express as px



In [2]:
# split input in dialog act and sentence
split_lines = []
with open('dialog_acts.dat', 'r') as f:  
    for line in f:
        split_lines.append(line.rstrip().split(' ', 1))
    
data = pd.DataFrame(split_lines, columns=['dialog_act', 'sentence'])

data.head()

,dialog_act,sentence
0,inform,im looking for a moderately priced restaurant ...
1,inform,any part of town
2,inform,bistro food
3,confirm,is there a moderately priced restaurant that s...
4,affirm,yes


In [3]:
x_train, x_test, y_train, y_test = train_test_split( data["sentence"], data["dialog_act"] , test_size=0.15, random_state=42)

In [4]:
def base_line1(data):
    return "inform"

In [5]:
rules = {
    "ack": ["okay", "uhm", "fine", "sure"],
    "affirm": ["yes", "right", "good", "do that", "agreed"],
    "bye": ["bye", "goodbye", "see you", "see you later"],
    "confirm": ["is it", "does it"],
    "deny": ["not", "dont"],
    "hello": ["hello", "hi"],
    "inform" : ["looking", "any", "doesn't matter"],
    "negate": ["no"],
    "null": ["cough"],
    "repeat": ["repeat"],
    "reqalts": ["what", "how"],
    "reqmore": ["more"],
    "request": ["address", "post", "what is", "postal"],
    "restart": ["start over"],
    "thankyou" : ["thank"],
}

def base_line2(x):
    for key, value in rules.items():
        for v in value:
            if v in x:
                return key
    return "null"

In [6]:
y_baseline1 = x_test.apply(base_line1)
y_baseline2 = x_test.apply(base_line2)

print("Baseline 1:")
print(classification_report(y_test, y_baseline1))

print("Baseline 2:")
print(classification_report(y_test, y_baseline2))

Baseline 1:


/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

              precision    recall  f1-score   support

         ack       0.00      0.00      0.00         5
      affirm       0.00      0.00      0.00       180
         bye       0.00      0.00      0.00        35
     confirm       0.00      0.00      0.00        22
        deny       0.00      0.00      0.00         6
       hello       0.00      0.00      0.00        14
      inform       0.40      1.00      0.57      1532
      negate       0.00      0.00      0.00        69
        null       0.00      0.00      0.00       232
      repeat       0.00      0.00      0.00         3
     reqalts       0.00      0.00      0.00       279
     reqmore       0.00      0.00      0.00         1
     request       0.00      0.00      0.00       972
     restart       0.00      0.00      0.00         2
    thankyou       0.00      0.00      0.00       474

    accuracy                           0.40      3826
   macro avg       0.03      0.07      0.04      3826
weighted avg       0.16   

In [7]:
deduped_data = data.drop_duplicates(subset='sentence')

x_train_deduped, x_test_deduped, y_train_deduped, y_test_deduped = train_test_split( deduped_data["sentence"], deduped_data["dialog_act"] , test_size=0.15, random_state=42)

In [26]:
count_vectorizer = CountVectorizer()
x_train_count = count_vectorizer.fit_transform(x_train_deduped)

pca = PCA(n_components=3)
x_train_pca = pca.fit_transform(x_train_count.toarray())

fig = px.scatter_3d(x=x_train_pca[:,0], y=x_train_pca[:,1], z=x_train_pca[:,2], color=y_train_deduped)

fig.show()

In [21]:
random_forest = LogisticRegression()

random_forest.fit(x_train_count, y_train_deduped)

x_test_count = count_vectorizer.transform(x_test_deduped)

y_pred = random_forest.predict(x_test_count)

print(classification_report(y_test_deduped, y_pred))

              precision    recall  f1-score   support

         ack       0.00      0.00      0.00         3
      affirm       1.00      0.83      0.91        24
         bye       1.00      0.56      0.71         9
     confirm       0.87      0.76      0.81        17
        deny       0.00      0.00      0.00         1
       hello       1.00      0.80      0.89         5
      inform       0.90      0.97      0.93       447
      negate       0.96      0.96      0.96        25
        null       0.57      0.26      0.36        46
      repeat       1.00      1.00      1.00         1
     reqalts       0.89      0.92      0.90        77
     request       0.94      0.95      0.94       135
     restart       1.00      1.00      1.00         1
    thankyou       0.92      0.85      0.88        13

    accuracy                           0.90       804
   macro avg       0.79      0.70      0.74       804
weighted avg       0.89      0.90      0.89       804



/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [20]:
random_forest = RandomForestClassifier()

random_forest.fit(x_train_count, y_train_deduped)

x_test_count = count_vectorizer.transform(x_test_deduped)

y_pred = random_forest.predict(x_test_count)

print(classification_report(y_test_deduped, y_pred))

              precision    recall  f1-score   support

         ack       0.00      0.00      0.00         3
      affirm       1.00      0.67      0.80        24
         bye       1.00      0.56      0.71         9
     confirm       0.83      0.88      0.86        17
        deny       0.00      0.00      0.00         1
       hello       1.00      0.40      0.57         5
      inform       0.89      0.95      0.92       447
      negate       1.00      0.96      0.98        25
        null       0.41      0.30      0.35        46
      repeat       1.00      1.00      1.00         1
     reqalts       0.89      0.84      0.87        77
     request       0.93      0.95      0.94       135
     restart       1.00      1.00      1.00         1
    thankyou       0.87      1.00      0.93        13

    accuracy                           0.88       804
   macro avg       0.77      0.68      0.71       804
weighted avg       0.87      0.88      0.88       804



/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/joel/uni/INFOMAIR/INFOMAIR_25/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

